<a href="https://colab.research.google.com/github/EugenioDiGaetano/NLP-Experiment/blob/main/wordPredictionsEmbedded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# Inizializzazione
```



In [30]:
import torch

# Simboli estratti dall'incipit della Divina Commedia
symbols = ["nel", "mezzo", "del", "cammin", "di", "nostra", "vita",
           "mi", "ritrovai", "per", "una", "selva", "oscura",
           "ché", "la", "diritta", "via", "era", "smarrita"]

# Creazione dei vettori one-hot
onehot_vectors = torch.eye(len(symbols))  # Matrice 14x14
#print (onehot_vectors.shape)


# Funzione per ottenere l'indice del simbolo
def i(symbol):
    return symbols.index(symbol)

# Parametri per la riduzione dimensionale
d = len(symbols)  # Dimensione originale (numero di simboli)
k = 3  # Dimensione ridotta
n = len(symbols)  # Numero di simboli
len_seq = 4

# Creazione della matrice di proiezione casuale per Johnson-Lindenstrauss
A = torch.randn(k, d) / torch.sqrt(torch.tensor(d, dtype=torch.float32))  # Proiezione con normalizzazione
#print (A.shape)
# Funzione per creare una sequenza one-hot concatenata
def seq(s1, s2, s3, s4):
    # Ottieni i vettori one-hot dei simboli
    one_hot = onehot_vectors[[i(s1), i(s2), i(s3), i(s4)]]
    # Proietta nello spazio ridotto con la matrice di proiezione
    reduced_seq = torch.matmul(one_hot, A.T).reshape(1, k*len_seq)  # Riduce la dimensione
    return reduced_seq

# Costruzione della matrice CMM (modificata con proiezione)
CMM = torch.matmul(seq("nel", "mezzo", "del", "cammin").T, onehot_vectors[[i("di")]]) +\
      torch.matmul(seq("mezzo", "del", "cammin", "di").T, onehot_vectors[[i("nostra")]]) +\
      torch.matmul(seq("del", "cammin", "di", "nostra").T, onehot_vectors[[i("vita")]]) +\
      torch.matmul(seq("cammin", "di", "nostra", "vita").T, onehot_vectors[[i("mi")]]) +\
      torch.matmul(seq("di", "nostra", "vita", "mi").T, onehot_vectors[[i("ritrovai")]]) +\
      torch.matmul(seq("nostra", "vita", "mi", "ritrovai").T, onehot_vectors[[i("per")]]) +\
      torch.matmul(seq("vita", "mi", "ritrovai", "per").T, onehot_vectors[[i("una")]]) +\
      torch.matmul(seq("mi", "ritrovai", "per", "una").T, onehot_vectors[[i("selva")]]) +\
      torch.matmul(seq("ritrovai", "per", "una", "selva").T, onehot_vectors[[i("oscura")]]) +\
      torch.matmul(seq("per", "una", "selva", "oscura").T, onehot_vectors[[i("ché")]]) +\
      torch.matmul(seq("una", "selva", "oscura", "ché").T, onehot_vectors[[i("la")]]) +\
      torch.matmul(seq("selva", "oscura", "ché", "la").T, onehot_vectors[[i("diritta")]]) +\
      torch.matmul(seq("oscura", "ché", "la", "diritta").T, onehot_vectors[[i("via")]]) +\
      torch.matmul(seq("ché", "la", "diritta", "via").T, onehot_vectors[[i("era")]]) +\
      torch.matmul(seq("la", "diritta", "via", "era").T, onehot_vectors[[i("smarrita")]])

print(CMM.shape)
# Predizione basata sul modello CMM
(s1, s2, s3, s4) = ("nel", "mezzo", "del", "cammin")
out = torch.matmul(seq(s1, s2, s3, s4), CMM)
predict_symb = symbols[torch.argmax(out)]

# Previsione iterativa
predicted_word = [s1, s2, s3, s4]
kkk=0
while True:
    out = torch.matmul(seq(s1, s2, s3, s4), CMM)
    s1, s2, s3 = s2, s3, s4
    s4 = symbols[torch.argmax(out)]
    kkk=kkk+1
    # Aggiungi la previsione alla lista
    predicted_word.append(s4)
    print (torch.argmax(out))
    # Se la previsione è un simbolo finale, fermati
    #if s4 == "smarrita":
    if kkk==15:
        break

print(predicted_word)


torch.Size([12, 19])
tensor(4)
tensor(5)
tensor(6)
tensor(7)
tensor(8)
tensor(9)
tensor(10)
tensor(11)
tensor(12)
tensor(13)
tensor(14)
tensor(15)
tensor(16)
tensor(17)
tensor(18)
['nel', 'mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per', 'una', 'selva', 'oscura', 'ché', 'la', 'diritta', 'via', 'era', 'smarrita']
